<a href="https://colab.research.google.com/github/Vishwajeetkaran/Sentiment-Analysis-Project/blob/main/SentimentAnalysisProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install seaborn -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


In [2]:
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import seaborn as sns
print("SeaBorn version:", sns.__version__)

from wordcloud import WordCloud
from tensorflow.keras import layers, models, losses, Sequential, optimizers, metrics

FileName = "/content/drive/MyDrive/Tweets.csv"

TensorFlow version: 2.15.0
SeaBorn version: 0.13.2


In [3]:
tweets_data = pd.read_csv(FileName)

tweets_data.sample(10)

,textID,text,selected_text,sentiment
685,5bae99f344,Didn`t get bitten by a rabies infested person ...,"! Gonna scare Rachel with Quarantine tonight, ...",neutral
2210,6b04f2321a,"FML dad didnt leave me any moneyz, so i have ...","FML dad didnt leave me any moneyz, so i have 2...",neutral
10028,a1cc1835fe,"First impression of Silverlight: sadly, based ...","sadly, based on what I saw today, it`s quite p...",negative
16603,7c1c80e54d,Friday tubes at work are slow..,Friday tubes at work are slow..,negative
25288,f914309bd5,must be time to go home - yes indeedy,must be time to go home - yes indeedy,neutral
27429,bb7af2712a,"I`ve heard of disgruntled investors, but call...",? That seems totally out of line,negative
13234,a8a1392eac,haha @ playing fall out boy in a bar full of o...,haha @ playing fall out boy in a bar full of o...,neutral
8932,528f5b2f92,i need some serious cheering up,i need some serious cheering up,negative
17299,83ea91fe2c,says ...tomorrow is another day... http://plu...,says ...tomorrow is another day...,neutral
13910,de0c6c9215,"Sore throat coming,and recording about to star...","Sore throat coming,and recording about to star...",negative


In [4]:
tweets_data = tweets_data.dropna()
tweets_data = tweets_data.drop_duplicates()

tweets_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27480 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB


In [5]:
tweets_data = tweets_data.drop(columns = ["selected_text", "textID"])

tweets_data.sample(10)

,text,sentiment
6620,my guys call me ``BartÂ ofÂ theÂ criticalÂ que...,positive
6034,hehe yeah! but too late hun,neutral
14954,"Dagnamit, latest power twitter update is pants...",neutral
13455,Feelin a little blue tonight.. Love might be good,neutral
9147,"Needless to say, I didn`t stay to find cache#90.",negative
8627,CONGRATS !! I just KNEW a princess would win!!...,positive
5841,Yea once - me and my friends flew out to amer...,neutral
19912,*moment of silence* for the Budget i`m bout to...,negative
22050,loves twitter (Y) 38th post. mwaha x,positive
18876,same,neutral


In [6]:
tweets_data["sentiment"].value_counts()

sentiment
neutral     11117
positive     8582
negative     7781
Name: count, dtype: int64

In [7]:
tweets_data["word_count"] = tweets_data["text"].apply(lambda x: len(str(x).split(" ")))

tweets_data[["text", "word_count"]].sample(10)

,text,word_count
23497,Helping my cousin get ready for her 8th grade ...,13
14362,Happy Star Wars Day! .... May the 4th Be with ...,11
14470,"Aaaagh, aircon not working in my office",7
24274,OHH yeah i sometimes forget your in the oth...,16
16035,yes! My co-workers don`t go to bars with me,10
21869,OPS sorry Queen Mom,5
17108,Fell off the wagon and had a Maccy`s!!,8
19984,"Not making good time, **** Chicago traffic - ...",11
15596,its ok. im juz curious,6
16192,workout stunk...no headphones.,3


In [8]:
from nltk.corpus import stopwords

nltk.download("stopwords")

stop_words = stopwords.words("english")

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
import string

def custom_standardization(input_data):

    # Convert to lowercase
    lowercase = tf.strings.lower(input_data)

    # Remove URLs
    stripped_urls = tf.strings.regex_replace(lowercase, r"https?://\S+|www\.\S+", "")

    # Remove email addresses
    stripped_symbol = tf.strings.regex_replace(stripped_urls, "\S*@\S*\s?", "")

    # Remove text in angular brackets (usually HTML tags)
    stripped_brackets = tf.strings.regex_replace(stripped_symbol, "<.*?>+", "")

    # Remove any square brackets and leave the text within square brackets
    stripped_brackets = tf.strings.regex_replace(stripped_brackets, "\[|\]", "")

    # Matches alphanumeric characters with digits and remove those
    stripped_digits = tf.strings.regex_replace(stripped_brackets, "\w*\d\w*", "")

    # Remove stopwords
    stripped_stopwords = tf.strings.regex_replace(stripped_digits, r"\b(?:{})\b".format("|".join(stop_words)), "")

    # Replace multiple whitespaces with a single whitespace
    stripped_whitespace_chars = tf.strings.regex_replace(stripped_stopwords, "\s+", " ")

    # Remove non-alphabet characters
    return tf.strings.regex_replace(stripped_whitespace_chars ,r"[^a-zA-Z\s]+" ,"")

In [10]:
tweets_data["cleaned_text"] = tweets_data["text"].apply(custom_standardization)

tweets_data

,text,sentiment,word_count,cleaned_text
0,"I`d have responded, if I were going",neutral,8,"tf.Tensor(b' responded going', shape=(), dtyp..."
1,Sooo SAD I will miss you here in San Diego!!!,negative,11,"tf.Tensor(b' sooo sad miss san diego', shape=(..."
2,my boss is bullying me...,negative,5,"tf.Tensor(b' boss bullying ', shape=(), dtype=..."
3,what interview! leave me alone,negative,6,"tf.Tensor(b' interview leave alone', shape=(),..."
4,"Sons of ****, why couldn`t they put them on t...",negative,15,tf.Tensor(b' sons put releases already bough...
...,...,...,...,...
27476,wish we could come see u on Denver husband l...,negative,18,tf.Tensor(b' wish could come see u denver husb...
27477,I`ve wondered about rake to. The client has ...,negative,26,tf.Tensor(b' wondered rake client made clear...
27478,Yay good for both of you. Enjoy the break - y...,positive,24,tf.Tensor(b' yay good enjoy break probably n...
27479,But it was worth it ****.,positive,8,"tf.Tensor(b' worth ', shape=(), dtype=string)"


In [11]:
# Define a function to convert tensor to string
def tensor_to_string(tensor):
    return tensor.numpy().decode("utf-8")  # Assuming utf-8 encoding

# Apply the function to the DataFrame column
tweets_data["cleaned_text"] = tweets_data["cleaned_text"].apply(tensor_to_string)
tweets_data["sentiment"] = tweets_data["sentiment"].replace({"negative": 0, "neutral": 1, "positive": 2})

tweets_data

,text,sentiment,word_count,cleaned_text
0,"I`d have responded, if I were going",1,8,responded going
1,Sooo SAD I will miss you here in San Diego!!!,0,11,sooo sad miss san diego
2,my boss is bullying me...,0,5,boss bullying
3,what interview! leave me alone,0,6,interview leave alone
4,"Sons of ****, why couldn`t they put them on t...",0,15,sons put releases already bought
...,...,...,...,...
27476,wish we could come see u on Denver husband l...,0,18,wish could come see u denver husband lost job...
27477,I`ve wondered about rake to. The client has ...,0,26,wondered rake client made clear net force...
27478,Yay good for both of you. Enjoy the break - y...,2,24,yay good enjoy break probably need hectic w...
27479,But it was worth it ****.,2,8,worth


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(tweets_data, test_size = 0.2, stratify = tweets_data["sentiment"], random_state = 123)
X_train, X_val = train_test_split(X_train, test_size = 0.1, stratify = X_train["sentiment"], random_state = 123)

X_train.shape, X_val.shape, X_test.shape

((19785, 4), (2199, 4), (5496, 4))

In [13]:
raw_train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train["text"].values, X_train["sentiment"].values)).shuffle(1024).batch(batch_size = 128)

raw_val_ds = tf.data.Dataset.from_tensor_slices(
    (X_val["text"].values, X_val["sentiment"].values)).batch(batch_size = 128)

raw_test_ds = tf.data.Dataset.from_tensor_slices(
    (X_test["text"].values, X_test["sentiment"].values)).batch(batch_size = 128)

len(raw_train_ds), len(raw_val_ds), len(raw_test_ds)

(155, 18, 43)

In [14]:
vocab_size = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length
)

In [15]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [16]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [17]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [18]:
lstm_model = models.load_model("/content/drive/MyDrive/sentiment_lstm.keras")
loss, accuracy = lstm_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

43/43 [==============================] - 20s 423ms/step - loss: 0.7501 - accuracy: 0.7056
Loss:  0.7501105070114136
Accuracy:  0.705604076385498


In [19]:
export_model = Sequential([
  vectorize_layer,
  lstm_model
])

export_model.compile(
    loss = losses.SparseCategoricalCrossentropy(from_logits = False),
    optimizer = optimizers.Adam(learning_rate = 0.001),
    metrics = ["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

43/43 [==============================] - 19s 407ms/step - loss: 0.7501 - accuracy: 0.7056
0.705604076385498


In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,\
                            classification_report

predictions = export_model.predict(X_test["text"].values)
y_pred = np.argmax(predictions , axis = 1)

y_test = X_test["sentiment"]
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average = None)
precision = precision_score(y_test,y_pred, average = None)
recall = recall_score(y_test,y_pred, average = None)

# Get the classification report
report = classification_report(y_test,y_pred)

# Print the classification report
print(report)

172/172 [==============================] - 39s 208ms/step
              precision    recall  f1-score   support

           0       0.72      0.65      0.69      1556
           1       0.67      0.71      0.69      2223
           2       0.75      0.75      0.75      1717

    accuracy                           0.71      5496
   macro avg       0.71      0.70      0.71      5496
weighted avg       0.71      0.71      0.71      5496



In [21]:
examples = [
  "this experience has been the worst, want my money back",
  "this AI related article is very good",
  "The movie was average",
  "its not good... its awesome!",
  "The weather is cloudy today"]

export_model.predict(examples)

1/1 [==============================] - 0s 113ms/step


array([[9.5829523e-01, 3.8627647e-02, 3.0770921e-03],
       [2.0512562e-02, 1.9357300e-01, 7.8591436e-01],
       [2.6830441e-01, 6.8715191e-01, 4.4543773e-02],
       [5.7421584e-04, 1.0368712e-02, 9.8905700e-01],
       [1.5866445e-01, 7.9768735e-01, 4.3648176e-02]], dtype=float32)

In [22]:
prediction = np.argmax(export_model.predict(examples), axis=1)
for idx, x in enumerate(prediction):
    dict = {0: "negative", 1: "neutral", 2: "positive"}
    print(dict[x], "{}%".format(round((export_model.predict(examples)[idx][x]*100),2)))

1/1 [==============================] - 0s 74ms/step
negative 95.83%
1/1 [==============================] - 0s 69ms/step
positive 78.59%
1/1 [==============================] - 0s 68ms/step
neutral 68.72%
1/1 [==============================] - 0s 73ms/step
positive 98.91%
1/1 [==============================] - 0s 71ms/step
neutral 79.77%
